In [36]:
import numpy as np
import pandas as pd
import csv
import matplotlib
from datetime import datetime
from math import *

def distance(lat1, lat2, lon1, lon2):
    lon1 = radians(float(lon1))
    lon2 = radians(float(lon2))
    lat1 = radians(float(lat1))
    lat2 = radians(float(lat2))

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371
    return (c * r)

def gradL_mse(phi, w, y):
    a = np.dot(phi.T, phi)
    a = pd.DataFrame(a)
    b = np.dot(phi.T, y)
    b = pd.DataFrame(b)
    c = np.dot(a, w)
    c = pd.DataFrame(c)
    return 2 * (c - b)

In [37]:
def normliz(phi, y):
    n = phi.shape[1]
    for i in range(1, n):
        phi.iloc[:, i] = (phi.iloc[:, i] - np.mean(phi.iloc[:, i])) / np.std(phi.iloc[:, i])
        #phi.iloc[:,i] = phi.iloc[:,i]/max(phi.iloc[:,i])
    phi = pd.DataFrame(phi)
    # y = (y - np.mean(y)) / np.std(phi[i])
    # y = (y - min(y)) / (max(y)-min(y))
    y = pd.DataFrame(y)
    return phi, y

def get_features(train,d):
    n = train.shape[0]
    dt = []
    for i in range(0, n):
        dt.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").timestamp())
    pick_long = []
    for i in range(0, n):
        pick_long.append(train.iloc[i, 1])
    pick_lat = []
    for i in range(0, n):
        pick_lat.append(train.iloc[i, 2])
    drop_long = []
    for i in range(0, n):
        drop_long.append(train.iloc[i, 3])
    drop_lat = []
    for i in range(0, n):
        drop_lat.append(train.iloc[i, 4])
    pass_count = []
    for i in range(0, n):
        pass_count.append(train.iloc[i, 5])
    const = np.ones(n)
    
    pick_long = list(map(round,pick_long)) 
    pick_lat = list(map(round,pick_lat)) 
    drop_long = list(map(round,drop_long)) 
    drop_lat = list(map(round,drop_lat)) 
    
    phi = [const, dt, pick_long, pick_lat, drop_long, drop_lat, pass_count]
    phi = pd.DataFrame(phi).T
    y = []
    for i in range(0, n):
        y.append(train.iloc[i, 6])
    y = list(map(log,list(map(log,y))))
    y = pd.DataFrame(y)
    [phi, y] = normliz(phi, y)
    return phi, y


def get_features_basis1_1(train,d):
    n = train.shape[0]
    dt = []
    for i in range(0, n):
        dt.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").timestamp())
    pick_long = []
    for i in range(0, n):
        pick_long.append(train.iloc[i, 1])
    pick_lat = []
    for i in range(0, n):
        pick_lat.append(train.iloc[i, 2])
    drop_long = []
    for i in range(0, n):
        drop_long.append(train.iloc[i, 3])
    drop_lat = []
    for i in range(0, n):
        drop_lat.append(train.iloc[i, 4])
    pass_count = []
    for i in range(0, n):
        pass_count.append(train.iloc[i, 5])
    weekends = []
    for i in range(0, n):
        if datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").weekday() < 5:
            weekends.append(0)
        else:
            weekends.append(1)
    month = []
    for i in range(0, n):
        month.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").month)
    const = np.ones(n)
    weekday = []
    for i in range(0, n):
        weekday.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").weekday())
    time = []
    for i in range(0, n):
        a = datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S")
        time.append(3600 * a.hour + 60 * a.minute + a.second)
    dist = []
    for i in range(0, n):
        dist.append(distance(pick_lat[i], drop_lat[i], pick_long[i], drop_long[i]))
    
    pick_long = list(map(round,pick_long)) 
    pick_lat = list(map(round,pick_lat)) 
    drop_long = list(map(round,drop_long)) 
    drop_lat = list(map(round,drop_lat)) 
    
    phi = [const, dt, pick_long, pick_lat, drop_long, drop_lat, dist]
    # time, month, dt,,d , pick_long, pick_lat, drop_long, drop_lat,
    phi = pd.DataFrame(phi).T
    [phi, y] = normliz(phi, phi[0])
    return phi


def get_features_basis1(train,d):
    n = train.shape[0]
    dt = []
    for i in range(0, n):
        dt.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").timestamp())
    pick_long = []
    for i in range(0, n):
        pick_long.append(train.iloc[i, 1])
    pick_lat = []
    for i in range(0, n):
        pick_lat.append(train.iloc[i, 2])
    drop_long = []
    for i in range(0, n):
        drop_long.append(train.iloc[i, 3])
    drop_lat = []
    for i in range(0, n):
        drop_lat.append(train.iloc[i, 4])
    pass_count = []
    for i in range(0, n):
        pass_count.append(train.iloc[i, 5])
    weekends = []
    for i in range(0, n):
        if datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").weekday() < 5:
            weekends.append(0)
        else:
            weekends.append(1)
    month = []
    for i in range(0, n):
        month.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").month)
    const = np.ones(n)
    weekday = []
    for i in range(0, n):
        weekday.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").weekday())
    time = []
    for i in range(0, n):
        a = datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S")
        time.append(3600 * a.hour + 60 * a.minute + a.second)
    dist=[]
    for i in range(0, n):
        dist.append(distance(pick_lat[i], drop_lat[i], pick_long[i], drop_long[i]))
    phi = [const, dt, pick_long, pick_lat, drop_long, drop_lat, dist]
    # weekday, time, month, dt,,d ,
    phi = pd.DataFrame(phi)
    phi = phi.T
    y = []
    for i in range(0, n):
        y.append(train.iloc[i, 6])
    y = list(map(log,list(map(log,y))))
    y = pd.DataFrame(y)
    [phi, y] = normliz(phi, y)
    return phi, y


# def get_features_basis2(train):
#     n = train.shape[0]
#     dt = []
#     for i in range(0, n):
#         dt.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").timestamp())
#     pick_long = []
#     for i in range(0, n):
#         pick_long.append(train.iloc[i, 1])
#     pick_lat = []
#     for i in range(0, n):
#         pick_lat.append(train.iloc[i, 2])
#     drop_long = []
#     for i in range(0, n):
#         drop_long.append(train.iloc[i, 3])
#     drop_lat = []
#     for i in range(0, n):
#         drop_lat.append(train.iloc[i, 4])
#     pass_count = []
#     for i in range(0, n):
#         pass_count.append(train.iloc[i, 5])
#     weekday = []
#     for i in range(0, n):
#         weekday.append(datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S").weekday())
#     time = []
#     for i in range(0, n):
#         a = datetime.strptime(train.iloc[i, 0][:-4], "%Y-%m-%d %H:%M:%S")
#         time.append(3600 * a.hour + 60 * a.minute + a.second)
#     const = np.ones(n)
#     phi = [const, weekday, time, dt, pick_long, pick_lat, drop_long, drop_lat, pass_count]
#     phi = pd.DataFrame(phi)
#     phi = phi.T
#     y = []
#     for i in range(0, n):
#         y.append(train.iloc[i, 6])
#     y = pd.DataFrame(y)
#     [phi, y] = normliz(phi, y)
#     return phi, y


def compute_RMSE(phi, w, y):
    error = (y - np.dot(phi, w))
    return np.linalg.norm(error)


def generate_output(phi_test, w):
    yp = np.matmul(phi_test, w)
    return yp


def closed_soln(phi, y):
    return np.linalg.pinv(phi).dot(y)


def gradient_descent(phi, y):
    n = phi.shape[1]
    lr = 0.000006
    w = np.random.rand(n, 1)
    w = pd.DataFrame(w)
    while np.linalg.norm(gradL_mse(phi, w, y)) > 0.01:
        w = w - lr * gradL_mse(phi, w, y)
    return w


def sgd(phi, y):
    n = phi.shape[1]
    lr = 0.00001
    w = np.random.rand(n, 1)
    phi = phi.to_numpy()
    y = y.to_numpy()
    while True:
        i = np.random.randint(phi.shape[0], size=1)
        grad = np.matmul(np.transpose(phi[i, :]), np.matmul(phi[i, :], w) - y[i])
        w = w - lr * grad
        # print(np.linalg.norm(grad))
        if np.linalg.norm(grad) < 0.00001:
            return w


def pnorm(phi, y, p):
    n = phi.shape[1]
    lr = 0.000001
    w = np.zeros(n)
    w = pd.DataFrame(w)
    lda = 1
    while np.linalg.norm(gradL_mse(phi, w, y) + p * lda * np.power(w, p - 1)) > 0.00001:
        w = w - lr * (gradL_mse(phi, w, y) + p * lda * np.power(w, p - 1))
        print(w)
    return w

In [38]:
train = pd.read_csv('train.csv')
train = train.iloc[:60000,:]
dev = pd.read_csv('dev.csv')
test = pd.read_csv('test.csv')


a = np.array(list(map(floor, train.pickup_latitude)))
b = np.array(list(map(floor, train.pickup_longitude)))
d = pd.DataFrame([a,b]).T
d['p'] = 0
c = pd.DataFrame([a,b,train.passenger_count]).T.groupby([0,1]).sum()
for i,j in zip(list(c.index),c.values):
    for k in range(np.shape(d)[0]):
        if d.iloc[k,0]==i[0] and d.iloc[k,1] == i[1] :
            d.iloc[k,-1] = j

d_train = d.iloc[:,-1]


a = np.array(list(map(floor, test.pickup_latitude)))
b = np.array(list(map(floor, test.pickup_longitude)))
d = pd.DataFrame([a,b]).T
d['p'] = 0
c = pd.DataFrame([a,b,test.passenger_count]).T.groupby([0,1]).sum()
for i,j in zip(list(c.index),c.values):
    for k in range(np.shape(d)[0]):
        if d.iloc[k,0]==i[0] and d.iloc[k,1] == i[1] :
            d.iloc[k,-1] = j

d_test = d.iloc[:,-1]


a = np.array(list(map(floor, dev.pickup_latitude)))
b = np.array(list(map(floor, dev.pickup_longitude)))
d = pd.DataFrame([a,b]).T
d['p'] = 0
c = pd.DataFrame([a,b,dev.passenger_count]).T.groupby([0,1]).sum()
for i,j in zip(list(c.index),c.values):
    for k in range(np.shape(d)[0]):
        if d.iloc[k,0]==i[0] and d.iloc[k,1] == i[1] :
            d.iloc[k,-1] = j
del a,b,c
d_dev = d.iloc[:,-1]
del d




In [39]:
#[phi_t, y_t] = get_features(train)
[phi_t1, y_t1] = get_features_basis1(train,d_train)
[phi_d1, y_d1] = get_features_basis1(dev,d_dev)
w_b1 = pnorm(phi_t1, y_t1, 2)
rmse_basis1 = compute_RMSE(phi_d1, w_b1, y_d1)

#[phi_t2, y_t2] = get_features_basis2(train)
#[phi_d2, y_d2] = get_features_basis2(dev)
#w_b2 = pnorm(phi_t2, y_t2, 2)
#rmse_basis2 = compute_RMSE(phi_d2, w_b2, y_d2)

print(rmse_basis1)
#print(rmse_basis2)

phi_test = get_features_basis1_1(test,d_test)
y = generate_output(phi_test.to_numpy(), w_b1.to_numpy())
np.savetxt('./sub.csv', y, delimiter=",")

          0
0  0.030562
1  0.001692
2  0.002506
3 -0.001462
4  0.002641
5 -0.001195
6  0.006390
          0
0  0.059902
1  0.003311
2  0.004798
3 -0.002821
4  0.005081
5 -0.002313
6  0.012443
          0
0  0.088068
1  0.004859
2  0.006892
3 -0.004081
4  0.007335
5 -0.003359
6  0.018179
          0
0  0.115107
1  0.006341
2  0.008802
3 -0.005250
4  0.009415
5 -0.004337
6  0.023616
          0
0  0.141065
1  0.007759
2  0.010540
3 -0.006332
4  0.011333
5 -0.005251
6  0.028772
          0
0  0.165985
1  0.009116
2  0.012119
3 -0.007335
4  0.013101
5 -0.006105
6  0.033664
          0
0  0.189907
1  0.010415
2  0.013551
3 -0.008261
4  0.014728
5 -0.006902
6  0.038306
          0
0  0.212873
1  0.011658
2  0.014846
3 -0.009117
4  0.016225
5 -0.007646
6  0.042714
          0
0  0.234920
1  0.012848
2  0.016014
3 -0.009907
4  0.017600
5 -0.008340
6  0.046900
          0
0  0.256085
1  0.013987
2  0.017064
3 -0.010635
4  0.018863
5 -0.008986
6  0.050878
          0
0  0.276403
1  0.015078
2  0

          0
0  0.748214
1  0.039543
2  0.010903
3 -0.012972
4  0.025179
5 -0.014915
6  0.139550
          0
0  0.748847
1  0.039576
2  0.010783
3 -0.012916
4  0.025114
5 -0.014889
6  0.139744
          0
0  0.749453
1  0.039607
2  0.010667
3 -0.012862
4  0.025051
5 -0.014863
6  0.139933
          0
0  0.750036
1  0.039637
2  0.010552
3 -0.012808
4  0.024989
5 -0.014838
6  0.140117
          0
0  0.750595
1  0.039666
2  0.010441
3 -0.012756
4  0.024928
5 -0.014813
6  0.140297
          0
0  0.751132
1  0.039694
2  0.010331
3 -0.012704
4  0.024869
5 -0.014789
6  0.140472
          0
0  0.751648
1  0.039720
2  0.010224
3 -0.012654
4  0.024810
5 -0.014765
6  0.140643
          0
0  0.752143
1  0.039746
2  0.010120
3 -0.012604
4  0.024753
5 -0.014741
6  0.140810
          0
0  0.752618
1  0.039771
2  0.010017
3 -0.012556
4  0.024697
5 -0.014718
6  0.140973
          0
0  0.753074
1  0.039794
2  0.009917
3 -0.012508
4  0.024643
5 -0.014695
6  0.141131
          0
0  0.753512
1  0.039817
2  0

          0
0  0.763601
1  0.040369
2  0.006307
3 -0.010723
4  0.022620
5 -0.013823
6  0.146656
          0
0  0.763618
1  0.040370
2  0.006292
3 -0.010715
4  0.022611
5 -0.013820
6  0.146679
          0
0  0.763634
1  0.040371
2  0.006277
3 -0.010707
4  0.022603
5 -0.013816
6  0.146701
          0
0  0.763649
1  0.040372
2  0.006262
3 -0.010700
4  0.022594
5 -0.013812
6  0.146722
          0
0  0.763664
1  0.040373
2  0.006248
3 -0.010693
4  0.022586
5 -0.013809
6  0.146744
          0
0  0.763678
1  0.040374
2  0.006235
3 -0.010686
4  0.022578
5 -0.013805
6  0.146764
          0
0  0.763692
1  0.040375
2  0.006221
3 -0.010679
4  0.022571
5 -0.013802
6  0.146784
          0
0  0.763705
1  0.040376
2  0.006208
3 -0.010672
4  0.022563
5 -0.013798
6  0.146804
          0
0  0.763718
1  0.040377
2  0.006196
3 -0.010665
4  0.022556
5 -0.013795
6  0.146823
          0
0  0.763730
1  0.040378
2  0.006183
3 -0.010659
4  0.022549
5 -0.013792
6  0.146842
          0
0  0.763741
1  0.040379
2  0

          0
0  0.764007
1  0.040401
2  0.005743
3 -0.010431
4  0.022293
5 -0.013680
6  0.147506
          0
0  0.764008
1  0.040401
2  0.005741
3 -0.010430
4  0.022292
5 -0.013680
6  0.147509
          0
0  0.764008
1  0.040401
2  0.005740
3 -0.010429
4  0.022291
5 -0.013679
6  0.147512
          0
0  0.764009
1  0.040401
2  0.005738
3 -0.010428
4  0.022290
5 -0.013679
6  0.147514
          0
0  0.764009
1  0.040401
2  0.005736
3 -0.010428
4  0.022289
5 -0.013678
6  0.147517
          0
0  0.764010
1  0.040401
2  0.005734
3 -0.010427
4  0.022288
5 -0.013678
6  0.147520
          0
0  0.764010
1  0.040402
2  0.005733
3 -0.010426
4  0.022287
5 -0.013678
6  0.147522
          0
0  0.764010
1  0.040402
2  0.005731
3 -0.010425
4  0.022286
5 -0.013677
6  0.147525
          0
0  0.764011
1  0.040402
2  0.005729
3 -0.010424
4  0.022285
5 -0.013677
6  0.147527
          0
0  0.764011
1  0.040402
2  0.005728
3 -0.010423
4  0.022284
5 -0.013676
6  0.147529
          0
0  0.764011
1  0.040402
2  0

          0
0  0.764019
1  0.040403
2  0.005673
3 -0.010395
4  0.022251
5 -0.013662
6  0.147613
          0
0  0.764019
1  0.040403
2  0.005673
3 -0.010395
4  0.022251
5 -0.013662
6  0.147613
          0
0  0.764019
1  0.040403
2  0.005673
3 -0.010394
4  0.022251
5 -0.013662
6  0.147613
          0
0  0.764019
1  0.040403
2  0.005672
3 -0.010394
4  0.022251
5 -0.013662
6  0.147614
          0
0  0.764019
1  0.040403
2  0.005672
3 -0.010394
4  0.022251
5 -0.013662
6  0.147614
          0
0  0.764019
1  0.040403
2  0.005672
3 -0.010394
4  0.022251
5 -0.013662
6  0.147614
          0
0  0.764019
1  0.040403
2  0.005672
3 -0.010394
4  0.022251
5 -0.013662
6  0.147615
          0
0  0.764019
1  0.040403
2  0.005671
3 -0.010394
4  0.022250
5 -0.013662
6  0.147615
          0
0  0.764019
1  0.040403
2  0.005671
3 -0.010394
4  0.022250
5 -0.013662
6  0.147615
          0
0  0.764019
1  0.040403
2  0.005671
3 -0.010394
4  0.022250
5 -0.013662
6  0.147616
          0
0  0.764019
1  0.040403
2  0

          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0.005664
3 -0.010390
4  0.022246
5 -0.013660
6  0.147626
          0
0  0.764019
1  0.040404
2  0

          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147627
          0
0  0.764019
1  0.040404
2  0

          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0

          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0.005663
3 -0.010390
4  0.022246
5 -0.013660
6  0.147628
          0
0  0.764019
1  0.040404
2  0